In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Please upvote if you find this notebook helpful 👍🏻
### 🔰 this is just a basic version, without any hypertuning! 🔰

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns




from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from scipy.stats import mode


from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.utils import to_categorical

from matplotlib import ticker
import time
import warnings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('float_format', '{:f}'.format)
warnings.filterwarnings('ignore')


RANDOM_STATE = 12 
FOLDS = 5

In [ ]:
df = pd.read_csv('../input/tabular-playground-series-feb-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-feb-2022/test.csv')
sample_submission = pd.read_csv('../input/tabular-playground-series-feb-2022/sample_submission.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
df['target'].value_counts()

In [ ]:
ax = sns.countplot(x=df['target'])


In [ ]:
TARGET = 'target'
FEATURES = [col for col in df.columns if col not in [TARGET]]

In [ ]:

cat_features = [col for col in df.columns if df[col].nunique() < 15]
cont_features = [col for col in df.columns if df[col].nunique() >= 15]

print(f'Total number of features: {len(FEATURES)}')
print(f'\033[92mNumber of categorical features: {len(cat_features)}')
print(f'\033[96mNumber of continuos features: {len(cont_features)}')

plt.pie([len(cat_features), len(cont_features)], 
        labels=['Categorical', 'Continuos'],
        colors=['#DE3163', '#58D68D'],
        textprops={'fontsize': 13},
        autopct='%1.1f%%')
plt.show()

In [ ]:
cat_features

In [ ]:
df['A0T0G10C0'].unique()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
y = df['target']
X = df[FEATURES]

In [ ]:
X_train, X_valid, y_train, y_valid = \
    train_test_split(X, y, stratify=y, train_size=0.75)


In [ ]:
model = RandomForestClassifier(n_estimators =250,max_depth=16,random_state=101)
model.fit(X_train,y_train)
predictions = model.predict(X_valid)

In [ ]:
y_valid.head(10)

In [ ]:
output = pd.DataFrame({'row_id':X_valid['row_id'],'y_valid':y_valid,'Predicted':predictions})
output.to_csv('my_submission.csv')

In [ ]:
submission = pd.read_csv('my_submission.csv')
submission.head(100)

In [ ]:
from sklearn.metrics import classification_report
from sklearn import metrics
import seaborn as sns
from sklearn.metrics import confusion_matrix

#Get the confusion matrix
cf_matrix = confusion_matrix(y_valid,predictions)

In [ ]:
unique = df['target'].unique()
print(unique)


In [ ]:
unique_sorted = sorted(unique)
unique_sorted

In [ ]:
plt.figure(figsize = (16,5))

ax = sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
      fmt='.2%',cmap='RdYlGn')


ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Bacteria Label')
ax.set_ylabel('Actual Bacteria Label ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['Bacteroides_fragilis',
 'Campylobacter_jejuni',
 'Enterococcus_hirae',
 'Escherichia_coli',
 'Escherichia_fergusonii',
 'Klebsiella_pneumoniae',
 'Salmonella_enterica',
 'Staphylococcus_aureus',
 'Streptococcus_pneumoniae',
 'Streptococcus_pyogenes'],rotation=90)


ax.yaxis.set_ticklabels(['Bacteroides_fragilis',
 'Campylobacter_jejuni',
 'Enterococcus_hirae',
 'Escherichia_coli',
 'Escherichia_fergusonii',
 'Klebsiella_pneumoniae',
 'Salmonella_enterica',
 'Staphylococcus_aureus',
 'Streptococcus_pneumoniae',
 'Streptococcus_pyogenes'],rotation=0)

## Display the visualization of the Confusion Matrix.


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize = (16,5))

ax = sns.heatmap(cf_matrix, annot=True,cmap='RdYlGn')

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Bacteria Label')
ax.set_ylabel('Actual Bacteria Label ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['Bacteroides_fragilis',
 'Campylobacter_jejuni',
 'Enterococcus_hirae',
 'Escherichia_coli',
 'Escherichia_fergusonii',
 'Klebsiella_pneumoniae',
 'Salmonella_enterica',
 'Staphylococcus_aureus',
 'Streptococcus_pneumoniae',
 'Streptococcus_pyogenes'],rotation=90)
ax.yaxis.set_ticklabels(['Bacteroides_fragilis',
 'Campylobacter_jejuni',
 'Enterococcus_hirae',
 'Escherichia_coli',
 'Escherichia_fergusonii',
 'Klebsiella_pneumoniae',
 'Salmonella_enterica',
 'Staphylococcus_aureus',
 'Streptococcus_pneumoniae',
 'Streptococcus_pyogenes'],rotation = 0)

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
X_valid.shape

In [ ]:
test.shape

In [ ]:
test.shape

In [ ]:
predictions2 = model.predict(test)
predictions2.shape

In [ ]:
submission2 = pd.read_csv('../input/tabular-playground-series-feb-2022/sample_submission.csv')
submission2.shape

In [ ]:
submission2 = pd.DataFrame({'row_id':submission2['row_id'],'target':predictions2})
submission2.to_csv('submission2.csv',index=False)

In [ ]:
submission = pd.read_csv('submission2.csv')

In [ ]:
sample_submission.head()

In [ ]:
submission2.head()

In [ ]:
import plotly.graph_objs as go

import plotly.offline as py

x, y = (list(x) for x in zip(*sorted(zip(model.feature_importances_, FEATURES), 
                                                            reverse = False)))
trace2 = go.Bar(
    x=x ,
    y=y,
    marker=dict(
        color=x,
        colorscale = 'Viridis',
        reversescale = True
    ),
    name='Random Forest Feature importance',
    orientation='h',
)

layout = dict(
    title='Barplot of Feature importances',
     width = 900, height = 2000,
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
#         domain=[0, 0.85],
    ),
    margin=dict(
    l=300,
),
)

fig1 = go.Figure(data=[trace2])
fig1['layout'].update(layout)
py.iplot(fig1, filename='plots')

## Please upvote if you find this notebook helpful 👍🏻